In [1]:
with open("out.tf", "r") as f:
    data = f.readlines()

In [2]:
EVENT, TIME, FROM_NODE, TO_NODE, PACKET_TYPE, PACKET_SIZE, FLAGS, FID, SOURCE_ADDRESS, DST_ADDRESS, SEQ_NUM, PKT_ID = [i for i in range(12)]

In [3]:
def filterPackets(data, condition):
    result = []
    for line in data:
        items = line.split()
        conditionSatisfied = True
        for key, value in condition.items():
            if items[key] != value:
                conditionSatisfied = False
                break
        if conditionSatisfied:
            result.append(items)
    return result

In [4]:
def getTotalPacketsSize(connection):
    return sum([int(packets[PACKET_SIZE]) for packets in connection])

In [5]:
tcp0Condition = {TO_NODE: '5', EVENT: 'r'}
tcp1Condition = {TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'tcp'}
tcp2Condition = {TO_NODE: '7', EVENT: 'r'}
udpCondition = {TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'cbr'}
ack0Condition = {FROM_NODE: '5', TO_NODE: '4', EVENT: '+'}
ack1Condition = {FROM_NODE: '6', TO_NODE: '4', EVENT: '+'}
ack2Condition = {FROM_NODE: '7', TO_NODE: '4', EVENT: '+'}
tcp0PacketSum = getTotalPacketsSize(filterPackets(data, tcp0Condition))
tcp1PacketSum = getTotalPacketsSize(filterPackets(data, tcp1Condition))
tcp2PacketSum = getTotalPacketsSize(filterPackets(data, tcp2Condition))
ack0PacketSum = getTotalPacketsSize(filterPackets(data, ack0Condition))
ack1PacketSum = getTotalPacketsSize(filterPackets(data, ack1Condition))
ack2PacketSum = getTotalPacketsSize(filterPackets(data, ack2Condition))
udpPacketSum = getTotalPacketsSize(filterPackets(data, udpCondition))
print("Average Throughput of TCP0:", (tcp0PacketSum + ack0PacketSum) / 10)
print("Average Throughput of TCP1:", (tcp1PacketSum + ack1PacketSum) / 10)
print("Average Throughput of TCP2:", (tcp2PacketSum + ack2PacketSum) / 10)
print("Average Throughput of UDP:", udpPacketSum / 1)

Average Throughput of TCP0: 110384.0
Average Throughput of TCP1: 38564.0
Average Throughput of TCP2: 35432.0
Average Throughput of UDP: 128000.0


In [6]:
def getDelayAndAmount(data, startCondition, endCondition):
    sentPackets = filterPackets(data, startCondition)
    startTime = {}
    for item in sentPackets:
        startTime[item[PKT_ID]] = float(item[TIME])
    receivedPackets = filterPackets(data, endCondition)
    totalDelay = 0
    for item in receivedPackets:
        totalDelay += float(item[TIME]) - startTime[item[PKT_ID]]
    return len(receivedPackets), totalDelay

In [8]:
tcp0Count, tcp0Delay = getDelayAndAmount(data, {FROM_NODE: '0', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '5', EVENT: 'r'})
ack0Count, ack0Delay = getDelayAndAmount(data, {FROM_NODE: '5', TO_NODE: '4', EVENT: '+'}, {FROM_NODE: '3', TO_NODE: '0', EVENT: 'r'})
print("Average Delay of TCP0:", (tcp0Delay + ack0Delay) / (tcp0Count + ack0Count))

tcp1Count, tcp1Delay = getDelayAndAmount(data, {FROM_NODE: '1', TO_NODE: '3', EVENT: '+', PACKET_TYPE: 'tcp'}, {FROM_NODE: '4', TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'tcp'})
ack1Count, ack1Delay = getDelayAndAmount(data, {FROM_NODE: '6', TO_NODE: '4', EVENT: '+'}, {FROM_NODE: '3', TO_NODE: '1', EVENT: 'r'})
print("Average Delay of TCP1:", (tcp1Delay + ack1Delay) / (tcp1Count + ack1Count))

tcp2Count, tcp2Delay = getDelayAndAmount(data, {FROM_NODE: '2', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '7', EVENT: 'r'})
ack2Count, ack2Delay = getDelayAndAmount(data, {FROM_NODE: '7', TO_NODE: '4', EVENT: '+'}, {FROM_NODE: '3', TO_NODE: '2', EVENT: 'r'})
print("Average Delay of TCP2:", (tcp0Delay + ack0Delay) / (tcp2Count + ack2Count))

udpCount, udpDelay = getDelayAndAmount(data, {FROM_NODE: '1', TO_NODE: '3', EVENT: '+', PACKET_TYPE: 'cbr'}, {FROM_NODE: '4', TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'cbr'})
print("Average Delay of UDP:", udpDelay / udpCount)

Average Delay of TCP0: 0.11958738074291253
Average Delay of TCP1: 0.12892446508379893
Average Delay of TCP2: 0.37184769148936025
Average Delay of UDP: 0.29460734375
